In [1]:
import scanpy as sc
import pandas as pd
import umap
import seaborn as sns
import matplotlib.pyplot as plt

from math import floor

from carot.utils import get_source_target_transport, score_transports_and_targets_combinations
from cmonge.utils import load_config, monge_get_source_target_transport
from cmonge.trainers.ot_trainer import MongeGapTrainer

from carot.trainers.conditional_monge_trainer import ConditionalMongeTrainer
from carot.datasets.conditional_loader import ConditionalDataModule
from carot.datasets.single_loader import CarModule

/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
default_colors = {
    "blue": "#648FFF",
    "purple": "#785EF0",
    "pink": "#DC267F",
    "orange": "#FE6100",
    "yellow": "#FFB000",
    "grey": "#DDDDDD",
}

ot_map = {
    "source": default_colors["blue"],
    "target": default_colors["pink"],
    "transport": default_colors["yellow"],
    "rest": default_colors["grey"]
}

# CMonge

In [3]:
base_ckpt_path = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/conditional-monge/experiments/cmonge/sel_CARs_LN_True_grad_acc_4_cosine/"
base_config_path = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/conditional-monge/experiments/cmonge/sel_CARs_LN_True_grad_acc_4_cosine/"
logger_path = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/conditional-monge/experiments/cmonge_ood/sel_CARs_ood/"
features = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/functional_and_score_genes.txt"


In [4]:
# setting = "avg_CAR"
# split = "ID" # ID or OOD

# for subset in ["CD4", "CD8"]:
#     adata = sc.read_h5ad(f'/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}_with_dummy_cars.h5ad')
#     adata.X = adata.layers['logcounts']
#     with open(features, "r") as f:
#         genes = f.read().splitlines()
#     sel_adata = adata[:, genes]

#     config_path = f"{base_config_path}{subset}_esm_XL_tail_dim_FuncScore/config.yaml"
#     config = load_config(config_path)
#     config.model.checkpointing_path = f"{base_ckpt_path}{subset}_esm_XL_tail_dim_FuncScore/model"
#     config.data.file_path = f"/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}_with_dummy_cars.h5ad"
#     config.data.features = features
#     config.data.drugs_path = f"/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}_sel_cars.txt"
#     config.model.embedding.model_dir = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/embedding/"
    
#     # Use average cars as source distribution
#     config.data.control_condition="sel_cars_mixed"
    
#     with open(config.data.drugs_path) as f:
#         train_cars = f.readlines()
#     train_cars = [c[:-1] for c in train_cars]

#     if split=="OOD":
#         config.ood_condition.split = [0,1,0]
#         config.data.split = [0,1,0]
#         config.condition.conditions = [car for car in sel_adata.obs["CAR_Variant"].unique() if not car in train_cars+["NA-NA-NA", 
#                                                                                                                       "NA-NA-z", 
#                                                                                                                       "sel_cars_mixed", 
#                                                                                                                       "all_cars_mixed", 
#                                                                                                                       "41BB_OOD_cars_mixed"]]
#     datamodule = ConditionalDataModule(config.data, config.condition, config.ae)
#     trainer = ConditionalMongeTrainer.load_checkpoint(
#         jobid=1,
#         logger_path=f"{logger_path}model_sel_cars_{subset}_{setting}_{split}_cars_eval.yaml",
#         config=config.model, 
#         datamodule=datamodule, 
#         ckpt_path=config.model.checkpointing_path
#         )

#     # Evaluate without transporting but using avg CAR as "prediction"
#     trainer.evaluate(datamodule, identity=True)

## UMAPs

In [5]:
base_ckpt_path = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/conditional-monge/experiments/cmonge/sel_CARs_LN_True_grad_acc_4_cosine/"
base_config_path = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/conditional-monge/experiments/cmonge/sel_CARs_LN_True_grad_acc_4_cosine/"
logger_path = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/conditional-monge/experiments/cmonge/sel_CARs_LN_True_grad_acc_4_cosine/avg_car_"
features = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/functional_and_score_genes.txt"
base_fig_save="/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Figures/transport_UMAPs/cmonge/sel_cars_"


In [6]:
for subset in ["CD4", "CD8"]:
    adata = sc.read_h5ad(f'/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}.h5ad')
    adata.X = adata.layers['logcounts']
    with open(features, "r") as f:
        genes = f.read().splitlines()
    sel_adata = adata[:, genes]
    umapper = umap.UMAP(random_state=1234)
    Y1 = umapper.fit_transform(sel_adata.X)
    sel_adata.obsm["X_umap"] = Y1
    
    config_path = f"{base_config_path}{subset}_esm_XL_tail_dim_FuncScore/config.yaml"
    config = load_config(config_path)
    config.model.checkpointing_path = f"{base_ckpt_path}{subset}_esm_XL_tail_dim_FuncScore/model"
    config.data.file_path = f"/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}_with_dummy_cars.h5ad"
    config.data.features = features
    config.data.drugs_path = f"/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}_sel_cars.txt"
    config.model.embedding.model_dir = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/embedding/"


    # avg_car_config = config.copy()
    # config.data.control_condition="sel_cars_mixed"
    # datamodule = ConditionalDataModule(config.data, config.condition, config.ae)
    # trainer = ConditionalMongeTrainer.load_checkpoint(
    #         jobid=1,
    #         logger_path=logger_path, 
    #         config=config.model, 
    #         datamodule=datamodule, 
    #         ckpt_path=config.model.checkpointing_path
    #         )
    adata = sc.read_h5ad(f'/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}_with_dummy_cars.h5ad')
    adata.X = adata.layers['logcounts']
    avg_adata = adata[adata.obs["CAR_Variant"]=="sel_cars_mixed", genes]
    avg_car_expr = pd.DataFrame(avg_adata.X.todense())
    avg_car_meta = avg_adata.obs.copy()

    Y_avg = umapper.transform(avg_car_expr)
    avg_car_meta[["UMAP1", "UMAP2"]] = Y_avg
    avg_car_meta["dtype"] = "transport"
    
    with open(config.data.drugs_path) as f:
        train_cars = f.readlines()
    train_cars = [c[:-1] for c in train_cars]
    print(sorted(train_cars))

    for car in sel_adata.obs["CAR_Variant"].unique():
        config.condition.split = [0.8,0.2,0]
        config.data.split = [0.8,0.2,0]
        if car in ["NA-NA-NA", "NA-NA-z", "sel_cars_mixed", "all_cars_mixed", "41BB_OOD_cars_mixed"]:
            continue
        print(car)
        if car not in train_cars:
            print("Changing split")
            config.condition.split = [0,1,0]
            config.data.split = [0,1,0]
        batch_size = min(sum(sel_adata.obs["CAR_Variant"]==car), sum(sel_adata.obs["CAR_Variant"]=="NA-NA-NA"))
        batch_size = floor(batch_size * config.data.split[1])
        config.data.batch_size=batch_size
        config.condition.conditions = [car]
        
        datamodule = ConditionalDataModule(config.data, config.condition, config.ae)
        trainer = ConditionalMongeTrainer.load_checkpoint(
            jobid=1,
            logger_path=logger_path, 
            config=config.model, 
            datamodule=datamodule, 
            ckpt_path=config.model.checkpointing_path
            )
    
        car_expr, car_meta = get_source_target_transport(datamodule=datamodule, 
                                                         trainer=trainer,
                                                         conditions=[car],
                                                        transport=False)
        sel_avg_meta = avg_car_meta.sample(batch_size)
        
        Y2 = umapper.transform(car_expr)
        car_meta[["UMAP1", "UMAP2"]] = Y2
        all_meta = pd.concat([car_meta, sel_avg_meta]).reset_index(drop=True)
    
        sns.scatterplot(x=Y1[:, 0], y=Y1[:, 1], color=default_colors["grey"], s=5, alpha=0.8, linewidth=0)
        sns.scatterplot(data=all_meta, x="UMAP1", y="UMAP2", hue="dtype", s=20, palette=ot_map, alpha=0.95, linewidth=0.15)
        plt.legend([],[], frameon=False)
        plt.tick_params(which="both", axis="both", bottom=False, left=False, labelbottom=False, labelleft=False)
        plt.xlabel("UMAP1", fontsize=18)
        plt.ylabel("UMAP2", fontsize=18)
        if car not in train_cars:
            plt.savefig(f"{base_fig_save}OOD_{subset}_{car}_avg_car_source_transport_target.png", bbox_inches="tight", dpi=300)
            plt.savefig(f"{base_fig_save}OOD_{subset}_{car}_avg_car_source_transport_target.pdf", bbox_inches="tight", dpi=300)
        else:
            plt.savefig(f"{base_fig_save}{subset}_{car}_source_avg_car_transport_target.png", bbox_inches="tight", dpi=300)
            plt.savefig(f"{base_fig_save}{subset}_{car}_source_avg_car_transport_target.pdf", bbox_inches="tight", dpi=300)
        plt.close()

/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/var/folders/02/gztwz1tj1051mtt_ykv6v7940000gn/T/ipykernel_25454/565742232.py:9: ImplicitModificationWarning: Setting element `.obsm['X_umap']` of view, initializing view as actual.
  sel_adata.obsm["X_umap"] = Y1
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


['41BB-41BB-z', '41BB-CD28-z', '41BB-CD40-z', '41BB-IL15RA-z', '41BB-NA-z', 'CD28-CD28-z', 'CD28-CD40-z', 'CD28-IL15RA-z', 'CD28-NA-z', 'CD40-IL15RA-z', 'CD40-NA-z', 'IL15RA-41BB-z', 'IL15RA-CD28-z', 'IL15RA-CD40-z', 'IL15RA-IL15RA-z', 'IL15RA-NA-z']
41BB-41BB-z


2025-08-22 19:35:03.060 | INFO     | cmonge.datasets.conditional_loader:setup_single_loader:83 - Setting up datamodules for 41BB-41BB-z
2025-08-22 19:35:03.728 | INFO     | cmonge.datasets.single_loader:splitter:80 - Splitting dataset started.
2025-08-22 19:35:03.875 | INFO     | cmonge.datasets.single_loader:splitter:121 - Target dataset number of cells - train: (746,), valid: (187,), test: (0,).
2025-08-22 19:35:03.876 | INFO     | cmonge.datasets.single_loader:splitter:124 - Control dataset number of cells - train: (607,), valid: (152,), test: (0,).
2025-08-22 19:35:03.876 | INFO     | cmonge.datasets.single_loader:splitter:127 - Splitting finished.
2025-08-22 19:35:03.876 | INFO     | cmonge.trainers.ot_trainer:__init__:42 - JAX uses cpu for trianing.
2025-08-22 19:35:04.963 | INFO     | cmonge.trainers.ot_trainer:load_checkpoint:139 - Loaded model from checkpoint
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: '

Evaluating on validation set
(151, 5120)
41BB-CD28-z


2025-08-22 19:35:07.078 | INFO     | cmonge.datasets.conditional_loader:setup_single_loader:83 - Setting up datamodules for 41BB-CD28-z
2025-08-22 19:35:07.768 | INFO     | cmonge.datasets.single_loader:splitter:80 - Splitting dataset started.
2025-08-22 19:35:07.774 | INFO     | cmonge.datasets.single_loader:splitter:121 - Target dataset number of cells - train: (1085,), valid: (272,), test: (0,).
2025-08-22 19:35:07.775 | INFO     | cmonge.datasets.single_loader:splitter:124 - Control dataset number of cells - train: (607,), valid: (152,), test: (0,).
2025-08-22 19:35:07.775 | INFO     | cmonge.datasets.single_loader:splitter:127 - Splitting finished.
2025-08-22 19:35:07.775 | INFO     | cmonge.trainers.ot_trainer:__init__:42 - JAX uses cpu for trianing.
2025-08-22 19:35:07.802 | INFO     | cmonge.trainers.ot_trainer:load_checkpoint:139 - Loaded model from checkpoint
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 

...

Evaluating on validation set
(234, 5120)
IL15RA-IL15RA-z
Changing split


2025-08-22 19:37:36.027 | INFO     | cmonge.datasets.conditional_loader:setup_single_loader:83 - Setting up datamodules for IL15RA-IL15RA-z
2025-08-22 19:37:36.656 | INFO     | cmonge.datasets.single_loader:splitter:80 - Splitting dataset started.
2025-08-22 19:37:36.668 | INFO     | cmonge.datasets.single_loader:splitter:121 - Target dataset number of cells - train: (0,), valid: (612,), test: (0,).
2025-08-22 19:37:36.668 | INFO     | cmonge.datasets.single_loader:splitter:124 - Control dataset number of cells - train: (0,), valid: (1423,), test: (0,).
2025-08-22 19:37:36.668 | INFO     | cmonge.datasets.single_loader:splitter:127 - Splitting finished.
2025-08-22 19:37:36.669 | INFO     | cmonge.trainers.ot_trainer:__init__:42 - JAX uses cpu for trianing.
2025-08-22 19:37:36.708 | INFO     | cmonge.trainers.ot_trainer:load_checkpoint:139 - Loaded model from checkpoint
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 

Evaluating on validation set
(612, 5120)
IL15RA-NA-z


2025-08-22 19:37:38.128 | INFO     | cmonge.datasets.conditional_loader:setup_single_loader:83 - Setting up datamodules for IL15RA-NA-z
2025-08-22 19:37:38.769 | INFO     | cmonge.datasets.single_loader:splitter:80 - Splitting dataset started.
2025-08-22 19:37:38.777 | INFO     | cmonge.datasets.single_loader:splitter:121 - Target dataset number of cells - train: (925,), valid: (232,), test: (0,).
2025-08-22 19:37:38.777 | INFO     | cmonge.datasets.single_loader:splitter:124 - Control dataset number of cells - train: (1138,), valid: (285,), test: (0,).
2025-08-22 19:37:38.777 | INFO     | cmonge.datasets.single_loader:splitter:127 - Splitting finished.
2025-08-22 19:37:38.778 | INFO     | cmonge.trainers.ot_trainer:__init__:42 - JAX uses cpu for trianing.
2025-08-22 19:37:38.804 | INFO     | cmonge.trainers.ot_trainer:load_checkpoint:139 - Loaded model from checkpoint
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 

Evaluating on validation set
(231, 5120)


# 41BB OOD

In [7]:
base_ckpt_path = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/conditional-monge/experiments/cmonge_ood/41BB_OOD/"
base_config_path = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/conditional-monge/experiments/cmonge_ood/41BB_OOD/"
logger_path = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/conditional-monge/experiments/cmonge_ood/41BB_OOD/"
features = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/functional_and_score_genes.txt"
base_fig_save="/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Figures/transport_UMAPs/41BB_ood/"


In [8]:
# setting = "avg_CAR"

# for subset in ["CD4", "CD8"]:
#     adata = sc.read_h5ad(f'/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}.h5ad')
#     adata.X = adata.layers['logcounts']
#     with open(features, "r") as f:
#         genes = f.read().splitlines()
#     sel_adata = adata[:, genes]

#     config_path = f"{base_config_path}{subset}/conditional-monge-cars-{subset}_41BB_OOD.yml"
#     config = load_config(config_path)
#     config.model.checkpointing_path = f"{base_ckpt_path}{subset}/model"
#     config.data.file_path = f"/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}_with_dummy_cars.h5ad"
#     config.data.features = features
#     config.data.drugs_path = f"/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/CAR_variants.txt"
#     config.model.embedding.model_dir = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/embedding/"

#     print(config.model.checkpointing_path)
    
#     config.condition.split = [0,1,0]
#     config.data.split = [0,1,0]

#     config.data.control_condition="41BB_OOD_cars_mixed"
#     config.condition.conditions = [car for car in sel_adata.obs["CAR_Variant"].unique() if "41BB" in car]
    
#     datamodule = ConditionalDataModule(config.data, config.condition, config.ae)
#     trainer = ConditionalMongeTrainer.load_checkpoint(
#         jobid=1,
#         logger_path=f"{logger_path}model_sel_cars_{subset}_{setting}_all_cars_eval.yaml",
#         config=config.model, 
#         datamodule=datamodule, 
#         ckpt_path=config.model.checkpointing_path
#         )

#     trainer.evaluate(datamodule, identity=True)


In [9]:
for subset in ["CD4", "CD8"]:
    adata = sc.read_h5ad(f'/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}.h5ad')
    adata.X = adata.layers['logcounts']
    with open(features, "r") as f:
        genes = f.read().splitlines()
    sel_adata = adata[:, genes]
    umapper = umap.UMAP(random_state=1234)
    Y1 = umapper.fit_transform(sel_adata.X)
    sel_adata.obsm["X_umap"] = Y1
    
    config_path = f"{base_config_path}{subset}/conditional-monge-cars-{subset}_41BB_OOD.yml"
    config = load_config(config_path)
    config.model.checkpointing_path = f"{base_ckpt_path}{subset}/model"
    config.data.file_path = f"/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}_with_dummy_cars.h5ad"
    config.data.features = features
    config.model.embedding.model_dir = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/embedding/"

    adata = sc.read_h5ad(f'/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}_with_dummy_cars.h5ad')
    adata.X = adata.layers['logcounts']
    avg_adata = adata[adata.obs["CAR_Variant"]=="41BB_OOD_cars_mixed", genes]
    avg_car_expr = pd.DataFrame(avg_adata.X.todense())
    avg_car_meta = avg_adata.obs.copy()

    Y_avg = umapper.transform(avg_car_expr)
    avg_car_meta[["UMAP1", "UMAP2"]] = Y_avg
    avg_car_meta["dtype"] = "transport"

    train_cars = [car for car in sel_adata.obs["CAR_Variant"].unique() if "41BB" in car]
    print(sorted(train_cars))

    for car in sel_adata.obs["CAR_Variant"].unique():
        config.condition.split = [0.8,0.2,0]
        config.data.split = [0.8,0.2,0]
        if car in ["NA-NA-NA", "NA-NA-z", "sel_cars_mixed", "all_cars_mixed", "41BB_OOD_cars_mixed"]:
            continue
        print(car)
        if car not in train_cars:
            print("Changing split")
            config.condition.split = [0,1,0]
            config.data.split = [0,1,0]
        batch_size = min(sum(sel_adata.obs["CAR_Variant"]==car), sum(sel_adata.obs["CAR_Variant"]=="NA-NA-NA"), len(avg_car_meta))
        batch_size = floor(batch_size * config.data.split[1])
        config.data.batch_size=batch_size
        config.condition.conditions = [car]
        
        datamodule = ConditionalDataModule(config.data, config.condition, config.ae)
        trainer = ConditionalMongeTrainer.load_checkpoint(
            jobid=1,
            logger_path=logger_path, 
            config=config.model, 
            datamodule=datamodule, 
            ckpt_path=config.model.checkpointing_path
            )
    
        car_expr, car_meta = get_source_target_transport(datamodule=datamodule, 
                                                         trainer=trainer,
                                                         conditions=[car],
                                                        transport=False)
        sel_avg_meta = avg_car_meta.sample(batch_size)
        
        Y2 = umapper.transform(car_expr)
        car_meta[["UMAP1", "UMAP2"]] = Y2
        all_meta = pd.concat([car_meta, sel_avg_meta]).reset_index(drop=True)
    
        sns.scatterplot(x=Y1[:, 0], y=Y1[:, 1], color=default_colors["grey"], s=5, alpha=0.8, linewidth=0)
        sns.scatterplot(data=all_meta, x="UMAP1", y="UMAP2", hue="dtype", s=20, palette=ot_map, alpha=0.95, linewidth=0.15)
        plt.legend([],[], frameon=False)
        plt.tick_params(which="both", axis="both", bottom=False, left=False, labelbottom=False, labelleft=False)
        plt.xlabel("UMAP1", fontsize=18)
        plt.ylabel("UMAP2", fontsize=18)
        if car not in train_cars:
            plt.savefig(f"{base_fig_save}OOD_{subset}_{car}_avg_car_source_transport_target.png", bbox_inches="tight", dpi=300)
            plt.savefig(f"{base_fig_save}OOD_{subset}_{car}_avg_car_source_transport_target.pdf", bbox_inches="tight", dpi=300)
        else:
            plt.savefig(f"{base_fig_save}{subset}_{car}_source_avg_car_transport_target.png", bbox_inches="tight", dpi=300)
            plt.savefig(f"{base_fig_save}{subset}_{car}_source_avg_car_transport_target.pdf", bbox_inches="tight", dpi=300)
        plt.close()

/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/var/folders/02/gztwz1tj1051mtt_ykv6v7940000gn/T/ipykernel_25454/1511633883.py:9: ImplicitModificationWarning: Setting element `.obsm['X_umap']` of view, initializing view as actual.
  sel_adata.obsm["X_umap"] = Y1
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


['41BB-41BB-z', '41BB-CD28-z', '41BB-CD40-z', '41BB-CTLA4-z', '41BB-IL15RA-z', '41BB-NA-z', 'CD28-41BB-z', 'CD40-41BB-z', 'CTLA4-41BB-z', 'IL15RA-41BB-z']
41BB-41BB-z


2025-08-22 19:38:06.977 | INFO     | cmonge.datasets.conditional_loader:setup_single_loader:83 - Setting up datamodules for 41BB-41BB-z
2025-08-22 19:38:07.292 | INFO     | cmonge.datasets.single_loader:splitter:80 - Splitting dataset started.
2025-08-22 19:38:07.298 | INFO     | cmonge.datasets.single_loader:splitter:121 - Target dataset number of cells - train: (746,), valid: (187,), test: (0,).
2025-08-22 19:38:07.298 | INFO     | cmonge.datasets.single_loader:splitter:124 - Control dataset number of cells - train: (607,), valid: (152,), test: (0,).
2025-08-22 19:38:07.298 | INFO     | cmonge.datasets.single_loader:splitter:127 - Splitting finished.
2025-08-22 19:38:07.298 | INFO     | cmonge.trainers.ot_trainer:__init__:42 - JAX uses cpu for trianing.
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/orbax/checkpoint/type_handlers.py:1421: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restorat

Evaluating on validation set
(151, 5120)
41BB-CD28-z


2025-08-22 19:38:08.244 | INFO     | cmonge.datasets.conditional_loader:setup_single_loader:83 - Setting up datamodules for 41BB-CD28-z
2025-08-22 19:38:08.569 | INFO     | cmonge.datasets.single_loader:splitter:80 - Splitting dataset started.
2025-08-22 19:38:08.574 | INFO     | cmonge.datasets.single_loader:splitter:121 - Target dataset number of cells - train: (1085,), valid: (272,), test: (0,).
2025-08-22 19:38:08.575 | INFO     | cmonge.datasets.single_loader:splitter:124 - Control dataset number of cells - train: (607,), valid: (152,), test: (0,).
2025-08-22 19:38:08.575 | INFO     | cmonge.datasets.single_loader:splitter:127 - Splitting finished.
2025-08-22 19:38:08.575 | INFO     | cmonge.trainers.ot_trainer:__init__:42 - JAX uses cpu for trianing.
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/orbax/checkpoint/type_handlers.py:1421: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restora

...

Evaluating on validation set
(1170, 5120)
IL15RA-IL15RA-z
Changing split


2025-08-22 19:40:36.322 | INFO     | cmonge.datasets.conditional_loader:setup_single_loader:83 - Setting up datamodules for IL15RA-IL15RA-z
2025-08-22 19:40:36.963 | INFO     | cmonge.datasets.single_loader:splitter:80 - Splitting dataset started.
2025-08-22 19:40:36.976 | INFO     | cmonge.datasets.single_loader:splitter:121 - Target dataset number of cells - train: (0,), valid: (612,), test: (0,).
2025-08-22 19:40:36.976 | INFO     | cmonge.datasets.single_loader:splitter:124 - Control dataset number of cells - train: (0,), valid: (1423,), test: (0,).
2025-08-22 19:40:36.976 | INFO     | cmonge.datasets.single_loader:splitter:127 - Splitting finished.
2025-08-22 19:40:36.977 | INFO     | cmonge.trainers.ot_trainer:__init__:42 - JAX uses cpu for trianing.
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/orbax/checkpoint/type_handlers.py:1421: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restora

Evaluating on validation set
(612, 5120)
IL15RA-NA-z
Changing split


2025-08-22 19:40:38.380 | INFO     | cmonge.datasets.conditional_loader:setup_single_loader:83 - Setting up datamodules for IL15RA-NA-z
2025-08-22 19:40:39.022 | INFO     | cmonge.datasets.single_loader:splitter:80 - Splitting dataset started.
2025-08-22 19:40:39.029 | INFO     | cmonge.datasets.single_loader:splitter:121 - Target dataset number of cells - train: (0,), valid: (1157,), test: (0,).
2025-08-22 19:40:39.030 | INFO     | cmonge.datasets.single_loader:splitter:124 - Control dataset number of cells - train: (0,), valid: (1423,), test: (0,).
2025-08-22 19:40:39.030 | INFO     | cmonge.datasets.single_loader:splitter:127 - Splitting finished.
2025-08-22 19:40:39.030 | INFO     | cmonge.trainers.ot_trainer:__init__:42 - JAX uses cpu for trianing.
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/orbax/checkpoint/type_handlers.py:1421: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restoratio

Evaluating on validation set
(1157, 5120)


# Monge - model per car

### Here the average car is based on ALL car variants

In [10]:
base_config_path = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/conditional-monge/experiments/monge/FuncScore/configs"
base_ckpt_path = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/conditional-monge/experiments/monge/FuncScore/checkpoints/NoEmbed_FuncScore_Monge_"
logger_path = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/conditional-monge/experiments/monge/FuncScore/avg_CAR_eval/"
features = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/functional_and_score_genes.txt"
base_fig_save="/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Figures/transport_UMAPs/monge/"


In [11]:
# setting = "avg_CAR"

# for subset in ["CD4", "CD8"]:
#     adata = sc.read_h5ad(f'/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}_with_dummy_cars.h5ad')
#     adata.X = adata.layers['logcounts']
#     with open(features, "r") as f:
#         genes = f.read().splitlines()
#     sel_adata = adata[:, genes]

#     for car in sel_adata.obs["CAR_Variant"].unique():
#         if car in ["NA-NA-NA", "NA-NA-z", "sel_cars_mixed", "all_cars_mixed", "41BB_OOD_cars_mixed"]:
#             continue
#         config_path = f"{base_config_path}/{subset}/NA-NA-NA_{car}_config.yaml"
#         config = load_config(config_path)
#         config.model.checkpointing_path = f"{base_ckpt_path}{subset}NA-NA-NA_{car}/"
#         config.data.file_path = f'/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}_with_dummy_cars.h5ad'
#         config.data.features = features
#         config.data.drugs_path = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/CAR_variants.txt"

#         config.data.control_condition="all_cars_mixed"
        
#         datamodule = CarModule(config.data)
#         trainer = MongeGapTrainer.load_checkpoint(
#                 jobid=1, config=config.model, logger_path=f"{logger_path}avg_cars_{subset}_{car}_eval.yaml"
#             )

#         trainer.evaluate(datamodule, identity=True, valid=True)

In [12]:
for subset in ["CD4", "CD8"]:
    adata = sc.read_h5ad(f'/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}.h5ad')
    adata.X = adata.layers['logcounts']
    with open(features, "r") as f:
        genes = f.read().splitlines()
    sel_adata = adata[:, genes]
    umapper = umap.UMAP(random_state=1234)
    Y1 = umapper.fit_transform(sel_adata.X)
    sel_adata.obsm["X_umap"] = Y1

    adata = sc.read_h5ad(f'/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}_with_dummy_cars.h5ad')
    adata.X = adata.layers['logcounts']
    avg_adata = adata[adata.obs["CAR_Variant"]=="all_cars_mixed", genes]
    avg_car_expr = pd.DataFrame(avg_adata.X.todense())
    avg_car_meta = avg_adata.obs.copy()

    Y_avg = umapper.transform(avg_car_expr)
    avg_car_meta[["UMAP1", "UMAP2"]] = Y_avg
    avg_car_meta["dtype"] = "transport"

    for car in sel_adata.obs["CAR_Variant"].unique():
        if car in ["NA-NA-NA", "NA-NA-z", "sel_cars_mixed", "all_cars_mixed", "41BB_OOD_cars_mixed"]:
            continue
        print(car)
        config_path = f"{base_config_path}/{subset}/NA-NA-NA_{car}_config.yaml"
        config = load_config(config_path)
        config.model.checkpointing_path = f"{base_ckpt_path}{subset}NA-NA-NA_{car}/"
        config.data.file_path = f"/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/{subset}_with_dummy_cars.h5ad"
        config.data.features = features
        config.data.drugs_path = "/Users/alicedriessen/Box/LegacyFromOldColleagues/Alice/CAR_Tcells/Model/OT/CAR_variants.txt"

        datamodule = CarModule(config.data)
        trainer = MongeGapTrainer.load_checkpoint(
                jobid=1, config=config.model, logger_path=logger_path
            )

        size = min(len(datamodule.control_valid_cells), len(datamodule.target_valid_cells), len(avg_car_meta))
        car_expr, car_meta = monge_get_source_target_transport(datamodule=datamodule, trainer=trainer, batch_size=size, transport=False)
        sel_avg_meta = avg_car_meta.sample(size)
        
        Y2 = umapper.transform(car_expr)
        car_meta[["UMAP1", "UMAP2"]] = Y2
        all_meta = pd.concat([car_meta, sel_avg_meta]).reset_index(drop=True)

        sns.scatterplot(x=Y1[:, 0], y=Y1[:, 1], color=default_colors["grey"], s=5, alpha=0.8, linewidth=0)
        sns.scatterplot(data=all_meta, x="UMAP1", y="UMAP2", hue="dtype", s=20, palette=ot_map, alpha=0.95, linewidth=0.15)
        plt.legend([],[], frameon=False)
        plt.tick_params(which="both", axis="both", bottom=False, left=False, labelbottom=False, labelleft=False)
        plt.xlabel("UMAP1", fontsize=18)
        plt.ylabel("UMAP2", fontsize=18)
        plt.savefig(f"{base_fig_save}{subset}_{car}_avg_car_model_source_transport_target.pdf", bbox_inches="tight", dpi=300)
        plt.savefig(f"{base_fig_save}{subset}_{car}_avg_car_model_source_transport_target.png", bbox_inches="tight", dpi=300)
        plt.close()

/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/var/folders/02/gztwz1tj1051mtt_ykv6v7940000gn/T/ipykernel_25454/2396592337.py:9: ImplicitModificationWarning: Setting element `.obsm['X_umap']` of view, initializing view as actual.
  sel_adata.obsm["X_umap"] = Y1
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


41BB-41BB-z


2025-08-22 19:41:09.039 | INFO     | cmonge.datasets.single_loader:splitter:80 - Splitting dataset started.
2025-08-22 19:41:09.046 | INFO     | cmonge.datasets.single_loader:splitter:121 - Target dataset number of cells - train: (746,), valid: (187,), test: (0,).
2025-08-22 19:41:09.046 | INFO     | cmonge.datasets.single_loader:splitter:124 - Control dataset number of cells - train: (607,), valid: (152,), test: (0,).
2025-08-22 19:41:09.046 | INFO     | cmonge.datasets.single_loader:splitter:127 - Splitting finished.
2025-08-22 19:41:09.046 | INFO     | cmonge.trainers.ot_trainer:__init__:42 - JAX uses cpu for trianing.
2025-08-22 19:41:09.612 | INFO     | cmonge.trainers.ot_trainer:load_checkpoint:139 - Loaded model from checkpoint
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Evaluating on validation set
41BB-CD28-z


2025-08-22 19:41:10.763 | INFO     | cmonge.datasets.single_loader:splitter:80 - Splitting dataset started.
2025-08-22 19:41:10.769 | INFO     | cmonge.datasets.single_loader:splitter:121 - Target dataset number of cells - train: (1085,), valid: (272,), test: (0,).
2025-08-22 19:41:10.769 | INFO     | cmonge.datasets.single_loader:splitter:124 - Control dataset number of cells - train: (607,), valid: (152,), test: (0,).
2025-08-22 19:41:10.784 | INFO     | cmonge.datasets.single_loader:splitter:127 - Splitting finished.
2025-08-22 19:41:10.784 | INFO     | cmonge.trainers.ot_trainer:__init__:42 - JAX uses cpu for trianing.
2025-08-22 19:41:10.793 | INFO     | cmonge.trainers.ot_trainer:load_checkpoint:139 - Loaded model from checkpoint
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

Evaluating on validation set
IL15RA-IL15RA-z


2025-08-22 19:42:40.216 | INFO     | cmonge.datasets.single_loader:splitter:80 - Splitting dataset started.
2025-08-22 19:42:40.222 | INFO     | cmonge.datasets.single_loader:splitter:121 - Target dataset number of cells - train: (489,), valid: (123,), test: (0,).
2025-08-22 19:42:40.222 | INFO     | cmonge.datasets.single_loader:splitter:124 - Control dataset number of cells - train: (1138,), valid: (285,), test: (0,).
2025-08-22 19:42:40.222 | INFO     | cmonge.datasets.single_loader:splitter:127 - Splitting finished.
2025-08-22 19:42:40.222 | INFO     | cmonge.trainers.ot_trainer:__init__:42 - JAX uses cpu for trianing.
2025-08-22 19:42:40.231 | INFO     | cmonge.trainers.ot_trainer:load_checkpoint:139 - Loaded model from checkpoint
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Evaluating on validation set
IL15RA-NA-z


2025-08-22 19:42:41.584 | INFO     | cmonge.datasets.single_loader:splitter:80 - Splitting dataset started.
2025-08-22 19:42:41.592 | INFO     | cmonge.datasets.single_loader:splitter:121 - Target dataset number of cells - train: (925,), valid: (232,), test: (0,).
2025-08-22 19:42:41.592 | INFO     | cmonge.datasets.single_loader:splitter:124 - Control dataset number of cells - train: (1138,), valid: (285,), test: (0,).
2025-08-22 19:42:41.592 | INFO     | cmonge.datasets.single_loader:splitter:127 - Splitting finished.
2025-08-22 19:42:41.592 | INFO     | cmonge.trainers.ot_trainer:__init__:42 - JAX uses cpu for trianing.
2025-08-22 19:42:41.600 | INFO     | cmonge.trainers.ot_trainer:load_checkpoint:139 - Loaded model from checkpoint
/Users/alicedriessen/.virtualenvs/carot/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Evaluating on validation set
